In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time 
from collections import defaultdict

plt.rcParams['font.sans-serif'] = ['Hiragino Sans GB', 'STHeiti', 'PingFang SC', 'Microsoft YaHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)

第一步：生成模拟数据
我们创建一些简单的二维数据，方便可视化理解

In [ ]:
def generate_sample_data(n_samples=1000, dim=2):
    '''
    生成3个明显分离的高斯分布簇
    loc是均值mean
    scale是标准差
    '''
    cluster1 = np.random.normal(loc=[2,2], scale=0.5, size=(n_samples//3, dim))
    cluster2 = np.random.normal(loc=[8,3], scale=0.6, size=(n_samples//3, dim))
    cluster3 = np.random.normal(loc=[5,8], scale=0.4, size=(n_samples - 2*(n_samples//3), dim))

    # vertical stack 将多个数组沿第一个轴
    data = np.vstack([cluster1, cluster2, cluster3])
    return data

In [4]:
data = generate_sample_data() 
print(f'数据形状: {data.shape}')

数据形状: (1000, 2)


第二步：实现K-means聚类

这是IVF算法的核心预处理步骤

In [ ]:
class SimpleKMeans:
    